# Pembelajaran Mesin: Bab 12 - Model Kustom dan Pelatihan dengan TensorFlow

**Selamat datang di Bab 12!** Pada bab ini, kita akan menyelami lebih dalam TensorFlow, melampaui API tingkat tinggi `tf.keras` yang telah kita gunakan sejauh ini. Kita akan mempelajari bagaimana TensorFlow memungkinkan kita untuk memiliki kontrol lebih besar atas model dan proses pelatihan, yang sangat berguna ketika kita membutuhkan fungsionalitas yang lebih spesifik atau eksperimental.

**Tujuan Pembelajaran Bab Ini:**
* Memahami arsitektur dasar TensorFlow.
* Mengenal dasar-dasar penggunaan TensorFlow seperti NumPy.
* Membuat fungsi loss, metrik, lapisan, dan model kustom.
* Mengimplementasikan *training loop* kustom.
* Memanfaatkan fitur AutoGraph TensorFlow untuk mempercepat komputasi.

---

## 1. Tur Singkat TensorFlow

TensorFlow adalah *library* komputasi numerik yang kuat, sangat cocok dan di-fine-tune untuk *Machine Learning* skala besar. Ini dikembangkan oleh tim Google Brain dan mendukung banyak layanan Google berskala besar.

### Fitur Utama TensorFlow:
* **Mirip NumPy dengan Dukungan GPU:** Inti TensorFlow sangat mirip dengan `NumPy ndarray`, tetapi dengan kemampuan untuk berjalan di GPU.
* **Komputasi Terdistribusi:** Mendukung komputasi terdistribusi di berbagai perangkat dan server.
* **Kompilator Just-In-Time (JIT):** Menyertakan sejenis kompilator *just-in-time* (JIT) yang memungkinkannya mengoptimalkan komputasi untuk kecepatan dan penggunaan memori. Ini bekerja dengan mengekstrak grafik komputasi dari fungsi Python, lalu mengoptimalkannya, dan akhirnya menjalankannya secara efisien (misalnya, dengan secara otomatis menjalankan operasi independen secara paralel).
* **Portabilitas Model:** Grafik komputasi dapat diekspor ke format portabel, memungkinkan model TensorFlow dilatih di satu lingkungan (misalnya, menggunakan Python di Linux) dan dijalankan di lingkungan lain (misalnya, menggunakan Java di perangkat Android).
* **Autodiff dan Optimizer:** Mengimplementasikan autodiff dan menyediakan beberapa *optimizer* yang sangat baik, seperti RMSProp dan Nadam, sehingga Anda dapat dengan mudah meminimalkan semua jenis fungsi *loss*.

### Arsitektur TensorFlow
Pada tingkat terendah, setiap operasi TensorFlow (disebut *op*) diimplementasikan menggunakan kode C++ yang sangat efisien. Banyak operasi memiliki banyak implementasi yang disebut *kernel*: setiap *kernel* didedikasikan untuk jenis perangkat tertentu, seperti CPU, GPU, atau bahkan TPU (*tensor processing units*).

**[Tambahkan Diagram Arsitektur TensorFlow dari Gambar 12-2, hal. 378]**

Sebagian besar waktu, kode Anda akan menggunakan API tingkat tinggi (terutama `tf.keras` dan `tf.data`). Namun, ketika Anda membutuhkan lebih banyak fleksibilitas, Anda akan menggunakan API Python tingkat rendah, menangani *tensor* secara langsung. TensorFlow berjalan tidak hanya di Windows, Linux, dan macOS, tetapi juga di perangkat seluler (menggunakan TensorFlow Lite), termasuk iOS dan Android. Bahkan ada implementasi JavaScript yang disebut TensorFlow.js yang memungkinkan model Anda berjalan langsung di *browser* Anda.

---

## 2. Menggunakan TensorFlow seperti NumPy

API TensorFlow berpusat pada *tensor*, yang mengalir dari satu operasi ke operasi lainnya—itulah asal nama TensorFlow. Sebuah *tensor* sangat mirip dengan `NumPy ndarray`: biasanya merupakan *array* multidimensi, tetapi juga dapat menampung *skalar* (nilai tunggal, seperti 42). *Tensor* ini akan penting ketika kita membuat fungsi *loss* kustom, metrik kustom, lapisan kustom, dan lainnya, jadi mari kita lihat cara membuat dan memanipulasinya.

### Tensor dan Operasi

Kita bisa membuat *tensor* dengan `tf.constant()`. Misalnya, berikut adalah *tensor* yang merepresentasikan matriks dengan dua baris dan tiga kolom *float*:

In [ ]:
import tensorflow as tf
import numpy as np

# Membuat tensor matriks
t_matrix = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("Tensor Matriks:\n", t_matrix)

# Membuat tensor skalar
t_scalar = tf.constant(42)
print("Tensor Skalar:\n", t_scalar)

# Mengakses shape dan dtype
print("Shape t_matrix:", t_matrix.shape)
print("Dtype t_matrix:", t_matrix.dtype)

Sama seperti `ndarray`, `tf.Tensor` memiliki `shape` dan tipe data (`dtype`). Pengindeksan (indexing) pada *tensor* bekerja mirip dengan NumPy:

In [ ]:
# Pengindeksan mirip NumPy
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("Slice t[:, 1:]:\n", t[:, 1:])
print("Slice t[..., 1, tf.newaxis]:\n", t[..., 1, tf.newaxis])

Yang paling penting, semua jenis operasi *tensor* tersedia:

In [ ]:
# Operasi tensor
print("t + 10:\n", t + 10)
print("tf.square(t):\n", tf.square(t))

# Perkalian matriks
print("t @ tf.transpose(t):\n", t @ tf.transpose(t))

Perhatikan bahwa menulis `t + 10` setara dengan memanggil `tf.add(t, 10)`. Operator lain seperti `-` dan `*` juga didukung. Operator `@` ditambahkan di Python 3.5, untuk perkalian matriks: ini setara dengan memanggil fungsi `tf.matmul()`.

**Catatan:** Banyak fungsi dan kelas memiliki alias (misalnya, `tf.add()` dan `tf.math.add()` adalah fungsi yang sama). Ini memungkinkan TensorFlow memiliki nama yang ringkas untuk operasi yang paling umum sambil mempertahankan paket yang terorganisir dengan baik.

### API Tingkat Rendah Keras
API Keras memiliki API tingkat rendah sendiri, yang terletak di `keras.backend`. Ini mencakup fungsi-fungsi seperti `square()`, `exp()`, dan `sqrt()`. Di `tf.keras`, fungsi-fungsi ini umumnya hanya memanggil operasi TensorFlow yang sesuai. Jika Anda ingin menulis kode yang akan portabel ke implementasi Keras lainnya, Anda harus menggunakan fungsi-fungsi Keras ini.

In [ ]:
from tensorflow import keras
K = keras.backend

t_keras = tf.constant([[1., 2.], [3., 4.]])
print("K.square(K.transpose(t_keras)) + 10:\n", K.square(K.transpose(t_keras)) + 10)

### Tensor dan NumPy

*Tensor* dan NumPy berinteraksi dengan baik: Anda dapat membuat *tensor* dari *array* NumPy, dan sebaliknya. Anda bahkan dapat menerapkan operasi TensorFlow ke *array* NumPy dan operasi NumPy ke *tensor*.

In [ ]:
a_np = np.array([2., 4., 5.])
print("tf.constant(a_np):\n", tf.constant(a_np))

t_tf = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("t_tf.numpy():\n", t_tf.numpy()) # atau np.array(t_tf)

print("tf.square(a_np):\n", tf.square(a_np))
print("np.square(t_tf):\n", np.square(t_tf))

**Penting:** NumPy menggunakan presisi 64-bit secara *default*, sedangkan TensorFlow menggunakan 32-bit. Ini karena presisi 32-bit umumnya lebih dari cukup untuk jaringan saraf, ditambah lagi berjalan lebih cepat dan menggunakan lebih sedikit RAM. Jadi, saat Anda membuat *tensor* dari *array* NumPy, pastikan untuk mengatur `dtype=tf.float32`.

### Konversi Tipe
Konversi tipe dapat sangat merusak kinerja, dan dapat dengan mudah tidak disadari ketika dilakukan secara otomatis. Untuk menghindarinya, TensorFlow tidak melakukan konversi tipe secara otomatis: ia hanya akan memberikan pengecualian jika Anda mencoba menjalankan operasi pada *tensor* dengan tipe yang tidak kompatibel. 

In [ ]:
try:
    tf.constant(2.) + tf.constant(40) # Akan menyebabkan error
except tf.errors.InvalidArgumentError as e:
    print(e)

# Gunakan tf.cast() untuk konversi tipe eksplisit
t2_tf64 = tf.constant(40., dtype=tf.float64)
print("tf.constant(2.0) + tf.cast(t2_tf64, tf.float32):\n", tf.constant(2.0) + tf.cast(t2_tf64, tf.float32))

### Variabel
Nilai `tf.Tensor` yang telah kita lihat sejauh ini bersifat *immutable* (tidak dapat diubah). Ini berarti kita tidak dapat menggunakan *tensor* biasa untuk mengimplementasikan bobot dalam jaringan saraf, karena perlu diubah dengan *backpropagation*. Ditambah lagi, parameter lain juga mungkin perlu berubah seiring waktu (misalnya, *momentum optimizer* melacak *gradient* sebelumnya). Yang kita butuhkan adalah `tf.Variable`:

In [ ]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print("Variabel v:\n", v)

# Variabel dapat dimodifikasi di tempat
v.assign(2 * v)
print("v setelah v.assign(2 * v):\n", v)

v[0, 1].assign(42)
print("v setelah v[0, 1].assign(42):\n", v)

v[:, 2].assign([0., 1.])
print("v setelah v[:, 2].assign([0., 1.]):\n", v)

v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
print("v setelah v.scatter_nd_update():\n", v)

Dalam praktiknya, Anda jarang perlu membuat variabel secara manual, karena Keras menyediakan metode `add_weight()` yang akan menanganinya untuk Anda. Ditambah lagi, parameter model umumnya akan diperbarui langsung oleh *optimizer*, jadi Anda jarang perlu memperbarui variabel secara manual.

### Struktur Data Lain
TensorFlow mendukung beberapa struktur data lain, antara lain:
* **Sparse Tensors (`tf.SparseTensor`):** Merepresentasikan *tensor* yang sebagian besar berisi nol secara efisien.
* **Tensor Arrays (`tf.TensorArray`):** Adalah daftar *tensor*. Mereka memiliki ukuran tetap secara *default* tetapi secara opsional dapat dibuat dinamis. Semua *tensor* yang mereka muat harus memiliki *shape* dan tipe data yang sama.
* **Ragged Tensors (`tf.RaggedTensor`):** Merepresentasikan daftar *list tensor* statis, di mana setiap *tensor* memiliki *shape* dan tipe data yang sama.
* **String Tensors:** *Tensor* biasa dengan tipe `tf.string`. Ini merepresentasikan *byte string*, bukan *Unicode string*.
* **Sets:** Direpresentasikan sebagai *tensor* biasa (atau *sparse tensor*). Misalnya, `tf.constant([[1, 2], [3, 4]])` merepresentasikan dua *set* `{1, 2}` dan `{3, 4}`.
* **Queues:** Menyimpan *tensor* di beberapa langkah. TensorFlow menawarkan berbagai jenis *queue*: *queue* *First In, First Out* (FIFO) sederhana (`FIFOQueue`), *queue* yang dapat memprioritaskan beberapa *item* (`PriorityQueue`), mengacak *item* mereka (`RandomShuffleQueue`), dan *batch item* dengan *shape* yang berbeda dengan *padding* (`PaddingFIFOQueue`).

---

## 3. Menyesuaikan Model dan Algoritma Pelatihan

Dengan *tensor*, operasi, variabel, dan berbagai struktur data yang tersedia, kita siap untuk menyesuaikan model dan algoritma pelatihan kita.

### Fungsi Loss Kustom (Custom Loss Functions)

Misalnya, kita ingin menggunakan *Huber loss* (diperkenalkan di Bab 10) sebagai fungsi *loss*. Meskipun sudah tersedia di `tf.keras`, kita akan mengimplementasikannya sendiri untuk demonstrasi. Cukup buat fungsi yang menerima *label* dan prediksi sebagai argumen, dan gunakan operasi TensorFlow untuk menghitung *loss* setiap *instance*:

In [ ]:
from tensorflow import keras

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Contoh penggunaan
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[1])])
model.compile(loss=huber_fn, optimizer="nadam")

# Data dummy untuk demonstrasi
X_train_dummy = tf.constant(np.random.rand(100, 1), dtype=tf.float32)
y_train_dummy = tf.constant(np.random.rand(100, 1), dtype=tf.float32)

model.fit(X_train_dummy, y_train_dummy, epochs=2)

Penting untuk mengembalikan *tensor* yang berisi satu *loss* per *instance* daripada *mean loss*, agar Keras dapat menerapkan *class weights* atau *sample weights* jika diperlukan.

### Menyimpan dan Memuat Model yang Mengandung Komponen Kustom

Menyimpan model yang berisi fungsi *loss* kustom berfungsi dengan baik, karena Keras menyimpan nama fungsi tersebut. Setiap kali Anda memuatnya, Anda perlu menyediakan kamus yang memetakan nama fungsi ke fungsi aktual.

In [ ]:
# Simpan model
model.save("my_model_with_a_custom_loss.h5")

# Muat model (dengan menyediakan fungsi kustom)
loaded_model = keras.models.load_model(
    "my_model_with_a_custom_loss.h5",
    custom_objects={"huber_fn": huber_fn}
)

Jika fungsi *loss* kustom Anda memiliki *hyperparameter* (misalnya, *threshold* untuk *Huber loss*), Anda harus membuat *subclass* dari `keras.losses.Loss` dan mengimplementasikan metode `get_config()` untuk menyimpan *hyperparameter* tersebut.

In [ ]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# Contoh penggunaan
model_huber_class = keras.models.Sequential([keras.layers.Dense(1, input_shape=[1])])
model_huber_class.compile(loss=HuberLoss(2.0), optimizer="nadam")
model_huber_class.fit(X_train_dummy, y_train_dummy, epochs=2)

# Simpan dan muat kembali
model_huber_class.save("my_model_huber_class.h5")
loaded_model_huber_class = keras.models.load_model(
    "my_model_huber_class.h5",
    custom_objects={"HuberLoss": HuberLoss}
)

### Fungsi Aktivasi Kustom, Initializer, Regularizer, dan Constraints

Sebagian besar fungsionalitas Keras, seperti *loss*, *regularizer*, *constraint*, *initializer*, metrik, fungsi aktivasi, lapisan, dan bahkan model lengkap, dapat disesuaikan dengan cara yang sama. Seringkali, Anda hanya perlu menulis fungsi sederhana dengan input dan output yang sesuai.

In [ ]:
# Fungsi aktivasi kustom
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

# Initializer kustom (Glorot normal)
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# Regularizer kustom (L1)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# Constraint kustom (hanya bobot positif)
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

# Contoh penggunaan dalam layer Dense
layer_custom = keras.layers.Dense(
    30,
    activation=my_softplus,
    kernel_initializer=my_glorot_initializer,
    kernel_regularizer=my_l1_regularizer,
    kernel_constraint=my_positive_weights
)

# Jika fungsi kustom memiliki hyperparameter yang perlu disimpan,
# subclass class yang sesuai (misalnya, keras.regularizers.Regularizer)
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

# Contoh penggunaan MyL1Regularizer
model_reg_custom = keras.models.Sequential([
    keras.layers.Dense(10, activation="relu", input_shape=[10]),
    keras.layers.Dense(1, activity_regularizer=MyL1Regularizer(0.01))
])
model_reg_custom.compile(loss="mse", optimizer="adam")
model_reg_custom.fit(tf.random.normal([100, 10]), tf.random.normal([100, 1]), epochs=1)

### Metrik Kustom

*Loss* dan metrik secara konseptual tidak sama: *loss* (misalnya, *cross entropy*) digunakan oleh *Gradient Descent* untuk melatih model, jadi harus *differentiable* (setidaknya di mana mereka dievaluasi), dan *gradient* mereka tidak boleh 0 di mana-mana. Sebaliknya, metrik (misalnya, akurasi) digunakan untuk mengevaluasi model: mereka harus lebih mudah diinterpretasikan, dan mereka bisa tidak *differentiable* atau memiliki *gradient* 0 di mana-mana.

Meskipun demikian, dalam kebanyakan kasus, mendefinisikan fungsi metrik kustom sama persis dengan mendefinisikan fungsi *loss* kustom. Namun, untuk metrik *streaming* (yang diperbarui secara bertahap di setiap *batch* seperti *precision* atau *recall*), Anda perlu membuat *subclass* dari `keras.metrics.Metric`.

In [ ]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = huber_fn # Menggunakan huber_fn yang sudah didefinisikan
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# Contoh penggunaan
model_metric = keras.models.Sequential([keras.layers.Dense(1, input_shape=[1])])
model_metric.compile(loss="mse", optimizer="nadam", metrics=[HuberMetric(2.0)])
model_metric.fit(X_train_dummy, y_train_dummy, epochs=2)

### Lapisan Kustom (Custom Layers)

Untuk membuat lapisan kustom tanpa bobot (misalnya, lapisan aktivasi), Anda dapat menggunakan `keras.layers.Lambda`. Contoh berikut adalah lapisan yang akan menerapkan fungsi eksponensial ke inputnya:

In [ ]:
# Layer kustom tanpa bobot
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

# Contoh penggunaan
model_lambda = keras.models.Sequential([
    keras.layers.Dense(10, input_shape=[10]),
    exponential_layer
])

Untuk membuat lapisan kustom dengan bobot, Anda perlu membuat *subclass* dari `keras.layers.Layer` dan mengimplementasikan metode `__init__()`, `build()`, `call()`, dan opsional `compute_output_shape()` serta `get_config()`.

In [ ]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # Harus di akhir build()

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

# Contoh penggunaan MyDense
model_my_dense = keras.models.Sequential([
    MyDense(10, activation="relu", input_shape=[10])
])

Jika lapisan Anda memiliki perilaku berbeda selama pelatihan dan pengujian (misalnya, menggunakan *Dropout* atau *BatchNormalization*), Anda harus menambahkan argumen `training` ke metode `call()`.

In [ ]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

# Contoh penggunaan
model_noise = keras.models.Sequential([
    keras.layers.Dense(10, input_shape=[10]),
    MyGaussianNoise(0.1)
])

### Model Kustom

Kita sudah membahas pembuatan kelas model kustom di Bab 10 dengan Subclassing API. Ini melibatkan *subclassing* kelas `keras.Model`, membuat lapisan dan variabel di konstruktor, dan mengimplementasikan metode `call()` untuk menentukan perilaku model.

In [ ]:
# Contoh Residual Block (dari buku)
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                           kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # Skip connection

# Contoh Model Kustom (dari buku)
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                           kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3): # 1+3 blocks (from the book's example)
            Z = self.block1(Z)
        Z = self.block2(Z) # This is where the second block is applied
        return self.out(Z)

# Menggunakan model kustom
model_custom = ResidualRegressor(1, input_shape=[8]) # input_shape here is for example
model_custom.compile(loss="mse", optimizer="adam")
# Latih model_custom dengan data yang sesuai
# model_custom.fit(X_train_data, y_train_data, epochs=...)

### Losses dan Metrik Berbasis Internal Model

Kadang-kadang, Anda mungkin ingin mendefinisikan *loss* atau metrik berdasarkan bagian lain dari model Anda, seperti bobot atau aktivasi lapisan tersembunyi. Ini dapat berguna untuk tujuan regularisasi atau untuk memantau aspek internal model Anda. Anda dapat mencapai ini dengan menghitung *loss* atau metrik tersebut di dalam metode `call()` model kustom Anda dan kemudian meneruskannya ke metode `add_loss()` atau `add_metric()`.

In [ ]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                           kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss) # Menambahkan reconstruction loss
        return self.out(Z)

# Contoh penggunaan
model_reconstruct = ReconstructingRegressor(1, input_shape=[8]) # input_shape here is for example
model_reconstruct.compile(loss="mse", optimizer="rmsprop")
# model_reconstruct.fit(X_train_data, X_train_data, epochs=...) # Latih dengan input sebagai target

---

## 4. Menghitung Gradient Menggunakan Autodiff

Untuk memahami cara menggunakan *autodiff* (dijelaskan di Bab 10 dan Lampiran D) untuk menghitung *gradient* secara otomatis, mari kita pertimbangkan fungsi mainan sederhana: `f(w1, w2) = 3 * w1 ** 2 + 2 * w1 * w2`.

In [ ]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

# Menggunakan tf.GradientTape untuk menghitung gradient
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print("Gradients:", gradients)

Secara *default*, *tape* akan secara otomatis dihapus segera setelah Anda memanggil metode `gradient()`-nya. Jika Anda perlu memanggil `gradient()` lebih dari sekali, Anda harus membuat *tape* *persistent* dan menghapusnya setiap kali Anda selesai dengannya untuk membebaskan sumber daya.

In [ ]:
# Tape persistent
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)
    
dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)
print("dz_dw1 (persistent):", dz_dw1)
print("dz_dw2 (persistent):", dz_dw2)
del tape # Penting untuk menghapus tape persistent setelah digunakan

Secara *default*, *tape* hanya akan melacak operasi yang melibatkan variabel. Namun, Anda dapat memaksa *tape* untuk mengawasi *tensor* apa pun yang Anda inginkan, untuk merekam setiap operasi yang melibatkannya.

In [ ]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients_watched = tape.gradient(z, [c1, c2])
print("Gradients (watched constants):", gradients_watched)

Untuk menghentikan *gradient* dari *backpropagating* melalui bagian tertentu dari jaringan, gunakan `tf.stop_gradient()`. Fungsi ini mengembalikan inputnya selama *forward pass*, tetapi tidak membiarkan *gradient* melewati selama *backpropagation*.

In [ ]:
def f_stop_gradient(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

w1_sg, w2_sg = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z_sg = f_stop_gradient(w1_sg, w2_sg)

gradients_sg = tape.gradient(z_sg, [w1_sg, w2_sg])
print("Gradients (with stop_gradient):", gradients_sg)

---

## 5. Perulangan Pelatihan Kustom (Custom Training Loops)

Dalam beberapa kasus yang jarang terjadi, metode `fit()` mungkin tidak cukup fleksibel untuk kebutuhan Anda (misalnya, menggunakan *optimizer* yang berbeda untuk bagian yang berbeda dari jaringan). Menulis *training loop* kustom akan membuat kode Anda lebih panjang, lebih rentan terhadap kesalahan, dan lebih sulit untuk dipelihara. Namun, ia memberi Anda kontrol penuh.

In [ ]:
# Dummy data untuk pelatihan
X_train_scaled = tf.constant(np.random.rand(100, 8), dtype=tf.float32)
y_train_scaled = tf.constant(np.random.rand(100, 1), dtype=tf.float32)

# Model sederhana
l2_reg = keras.regularizers.l2(0.05)
model_custom_loop = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg, input_shape=[8]),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

# Fungsi untuk mengambil batch acak
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

# Fungsi untuk menampilkan status bar (versi sederhana)
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join([" {}: {:.4f}".format(m.name, m.result())
                          for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} -".format(iteration, total) + metrics, end=end)

# Hyperparameter pelatihan
n_epochs = 5
batch_size = 32
n_steps = len(X_train_scaled) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

# Perulangan pelatihan kustom
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train_scaled)
        with tf.GradientTape() as tape:
            y_pred = model_custom_loop(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model_custom_loop.losses)
        
        gradients = tape.gradient(loss, model_custom_loop.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_custom_loop.trainable_variables))
        
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        
        print_status_bar(step * batch_size, len(X_train_scaled), mean_loss, metrics)
    
    print_status_bar(len(X_train_scaled), len(X_train_scaled), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

---

## 6. Fungsi dan Grafik TensorFlow

Di TensorFlow 1, grafik tidak dapat dihindari (begitu juga dengan kompleksitas yang menyertainya) karena mereka adalah bagian sentral dari API TensorFlow. Di TensorFlow 2, mereka masih ada, tetapi tidak terlalu sentral, dan jauh lebih sederhana untuk digunakan. Untuk menunjukkan betapa sederhananya, mari kita mulai dengan fungsi trivial yang menghitung pangkat tiga dari inputnya:

In [ ]:
@tf.function
def tf_cube(x):
    print("x =", x) # Ini hanya akan dieksekusi saat tracing
    return x ** 3

print("Hasil tf_cube(2.0):", tf_cube(tf.constant(2.0)))
print("Hasil tf_cube(3.0):", tf_cube(tf.constant(3.0))) # Reuses existing graph
print("Hasil tf_cube(2):", tf_cube(2)) # New Python value, new graph is traced

Di bawah kap, `tf.function()` menganalisis komputasi yang dilakukan oleh fungsi `cube()` dan menghasilkan *computation graph* yang setara. TensorFlow mengoptimalkan *computation graph*, memangkas *node* yang tidak digunakan, menyederhanakan ekspresi (misalnya, `1 + 2` akan diganti dengan `3`), dan lainnya. Setelah *graph* yang dioptimalkan siap, *TF Function* secara efisien mengeksekusi operasi dalam *graph*, dalam urutan yang sesuai (dan secara paralel jika bisa). Sebagai hasilnya, *TF Function* biasanya akan berjalan jauh lebih cepat daripada fungsi Python aslinya, terutama jika melakukan komputasi yang kompleks.

### AutoGraph dan Tracing

Bagaimana TensorFlow menghasilkan grafik? Ini dimulai dengan menganalisis kode sumber fungsi Python untuk menangkap semua pernyataan kontrol aliran, seperti loop `for`, loop `while`, dan pernyataan `if`, serta pernyataan `break`, `continue`, dan `return`. Langkah pertama ini disebut *AutoGraph*. Setelah menganalisis kode fungsi, *AutoGraph* menghasilkan versi fungsi yang ditingkatkan di mana semua pernyataan kontrol aliran diganti dengan operasi TensorFlow yang sesuai.

**[Tambahkan Diagram AutoGraph dan Tracing dari Gambar 12-4, hal. 408]**

`for` loop hanya akan ditangkap sebagai *loop* dinamis dalam *graph* jika berulang pada `tf.range()`, bukan `range()`. Ini memberi Anda pilihan:

In [ ]:
@tf.function
def add_10_dynamic(x):
    for i in tf.range(10): # Menggunakan tf.range
        x += 1
    return x

print("Hasil add_10_dynamic(0):", add_10_dynamic(tf.constant(0)))
# Periksa operasi graph untuk melihat While op
# print(add_10_dynamic.get_concrete_function(tf.constant(0)).graph.get_operations())

### Aturan Fungsi TF
Sebagian besar waktu, mengubah fungsi Python yang melakukan operasi TensorFlow menjadi *TF Function* sangatlah mudah: cukup *decorate* dengan `@tf.function` atau biarkan Keras menanganinya untuk Anda. Namun, ada beberapa aturan yang harus dipatuhi:
* **Operasi TensorFlow:** Jika Anda memanggil *library* eksternal apa pun, termasuk NumPy atau bahkan *standard library*, panggilan ini hanya akan berjalan selama *tracing*; itu tidak akan menjadi bagian dari *graph*. Jadi, pastikan Anda menggunakan `tf.reduce_sum()` daripada `np.sum()`, `tf.sort()` daripada fungsi bawaan `sorted()`, dan seterusnya (kecuali Anda benar-benar ingin kode tersebut hanya berjalan selama *tracing*).
* **Efek Samping:** Jika kode non-TensorFlow Anda memiliki efek samping (misalnya, mencatat sesuatu atau memperbarui *counter* Python), maka Anda tidak boleh berharap efek samping tersebut terjadi setiap kali Anda memanggil *TF Function*, karena itu hanya akan terjadi saat fungsi tersebut di-*trace*.
* **Pembuatan Variabel:** Jika fungsi membuat variabel TensorFlow (atau objek TensorFlow *stateful* lainnya, seperti *dataset* atau *queue*), ia harus melakukannya pada panggilan pertama, dan hanya pada saat itu, jika tidak, Anda akan mendapatkan pengecualian. Umumnya lebih baik membuat variabel di luar *TF Function* (misalnya, di metode `build()` dari lapisan kustom). Jika Anda ingin menetapkan nilai baru ke variabel, pastikan Anda memanggil metode `assign()`-nya, daripada menggunakan operator `=`.
* **Kode Sumber:** Kode sumber fungsi Python Anda harus tersedia untuk TensorFlow. Jika kode sumber tidak tersedia, maka proses pembuatan *graph* akan gagal atau memiliki fungsionalitas terbatas.
* **Loop Dinamis:** TensorFlow hanya akan menangkap loop `for` yang berulang pada *tensor* atau *dataset*. Jadi pastikan Anda menggunakan `for i in tf.range(x)` daripada `for i in range(x)`, jika tidak, loop tersebut tidak akan ditangkap dalam *graph*.
* **Vektorisasi:** Seperti biasa, untuk alasan kinerja, Anda harus lebih memilih implementasi *vectorized* kapan pun Anda bisa, daripada menggunakan *loop*.

---

**Demikianlah kerangka Jupyter Notebook untuk Bab 12.** Anda dapat mengisi bagian "dummy data" dengan data yang lebih relevan (misalnya, dari dataset California Housing atau Fashion MNIST) dan menambahkan visualisasi serta analisis tambahan untuk memperdalam pemahaman Anda. Pastikan untuk menjalankan kode secara bertahap dan mengamati keluarannya.